# EXTER model implementation in pyomo


https://www.pep-net.org/sites/pep-net.org/files/EXTER-en.pdf

## 1 Preparation

### 1.1 Load libraries

In [ ]:
import pyomo.environ as pyo
from pyomo.environ import AbstractModel, Set, Param, Var, Constraint
import pyomo.dataportal.DataPortal as DataPortal
import pandas as pd
import json
from pyomo.core.expr.numvalue import value as constraintValue
import idaes.core.util as idaes_utils
from pyomo.core.expr.current import identify_variables
from pyomo.common.collections import ComponentSet
import numpy as np
from openpyxl import load_workbook

### 1.2 Rewrite data

In [ ]:
# df = pd.read_excel('data/EXTER.xlsx', sheet_name='EXTER', 
# header=[0,1], index_col=[0,1])
df = pd.read_excel('data/EXTER.xlsx', 
    sheet_name='EXTER', header=[0,1], index_col=[0,1], skiprows=0)
df.to_json('data/EXTER.json')

### 1.3 Helper functions

In [ ]:
sam_dict = {}

## Make some helper functions
## Dictionary is organizec as "FROM (C, D) TO (A, B)"
## But commands will come in form (TO (A, B) FROM (C, D))
def sam(c, over=[], prices=None):
    
    coord = [0,0,0,0]
    d = {}
    p = {}
    if prices:
        if isinstance(prices, int):
            if(len(over) == 1):
                for cat in over[0]:
                    p[cat] = prices
            elif(len(over) == 2):
                for cat in over[1]:
                    p[cat] = prices
            elif (len(over) == 0):
                p['singular'] = prices
        else:
            p = prices
    p['ADM'] = 1 #hack
    
    # print(p)
    ## need to loop over levels:
    if (len(over) == 0):
        # print(sam_dict[f"('{c[0]}', '{c[1]}')"][f"('{c[2]}', '{c[3]}')"])
        if ('singular' in list(p.keys())):
            return sam_dict[f"('{c[2]}', '{c[3]}')"][f"('{c[0]}', '{c[1]}')"] / p['singular']    
        else:
            return sam_dict[f"('{c[2]}', '{c[3]}')"][f"('{c[0]}', '{c[1]}')"]
        # return sam[(c[0],c[1])][(c[2],c[3])]
        ## return the coordinate
    if len(over) == 1:
        idx = c.index(0)
        for cat in over[0]:
            coord = c.copy()
            coord[idx] = cat
            d[cat] = sam(coord)
            if d[cat] is None: d[cat] = 0 #fix nonetype values
            ## adjust for prices
            if cat in p.keys():
                d[cat] = d[cat]/p[cat]
    if len(over) == 2:
        idx = c.index(0)
        idx2 = c.index(1)
        for cat in over[0]:
            for cat2 in over[1]:
                coord = c.copy()
                coord[idx] = cat
                coord[idx2] = cat2
                d[(cat,cat2)] = sam(coord)
                if d[(cat,cat2)] is None: d[(cat,cat2)] = 0 #fix nonetype values
                ## assume only 2nd variable is adjusted for prices
                if cat in p.keys():
                    d[(cat,cat2)] = d[(cat,cat2)]/p[cat]
                
    


    # print(d)
    return d

## helper function for retriving initial values
def val(obj, sub1=None, sub2=None, default=None):
    try:
        if isinstance(obj, Var):
            if (sub2): return obj._value_init_value[(sub1,sub2)]
            if (sub1): return obj._value_init_value[sub1]
            return obj._value_init_value
        if isinstance(obj, Param):
            values = obj.default()
            if (sub2): return values[(sub1,sub2)]
            if (sub1): return values[sub1]
            return values
    except KeyError:
        if default is not None:
            return default
        else:
            raise


def divide(d1, d2):
    d = {}
    for key in d1.keys():
        d[key] = d1[key]/d2[key]
    return d


### 1.4 Initialize model

In [ ]:
m = AbstractModel()

## 2 Define sets

In [ ]:
sectors = ['AGR','MAN','SER','PUB']
tradeables = ['AGR','MAN','SER']
goods = ['AGR','MAN']
households = ['SAL', 'CAP']
m.sectors = Set(dimen=1,initialize=sectors, doc='Industries and commodities') 
m.tradeables = Set(dimen=1, initialize=tradeables, doc='Tradeable commodities')
m.goods = Set(dimen=1, initialize=goods, doc='Goods')
m.households = Set(dimen=1, initialize=households, doc='Households')

## 3 Load data

In [ ]:
sam_dict = DataPortal()
sam_dict.load(filename='data/EXTER.json', encoding='utf-8')
sam_dict.data()

## 4 Variables and parameters

### 4.1 Prices

In [ ]:
#note! structure here is different: FROM (AG, FIRM) TO (AG, CAP)
# the GAMS code has it the other way around!

# Let's get prices first:
prices = {
    'e': 1,
    'W': 1,
    'PINDEX': 1,
    'R': { 'AGR': 1, 'MAN': 1, 'SER': 1 },
    'PL': { 'AGR': 1, 'MAN': 1, 'SER': 1 },
    'PE': { 'AGR': 1, 'MAN': 1, 'SER': 1 },
    'PWM': { 'AGR': 1, 'MAN': 1, 'SER': 1 },
    'P': { 'AGR': 1, 'MAN': 1, 'SER': 1, 'PUB': 1 }
}


## Get some nominal variables
TI_nominal = sam(['AG', 'GVT','I', 0], over=[tradeables]) #"Receipts from indirect tax on commodity tr"
DD_nominal = sam(['J', 0, 'I', 1, ], over=[tradeables, tradeables])  #"Demand for domestic commodity tr"
IM_nominal = sam(['AG', 'WORLD','I', 0], over=[tradeables]) #"Imports of commodity tr"
DI_nominal = sam(['I', 0, 'J', 1], over=[tradeables, sectors])
CI_nominal = dict(map(lambda j: (j, 
    sum([DI_nominal[(tr, j)] for tr in tradeables])
    ), sectors))


## Calculate some parameters
tm = divide(sam(['AG','DUTIES','I',0], over=[tradeables]), sam(['AG','WORLD','I',0], over=[tradeables]))
tx = dict(map(lambda tr: (tr, 
    TI_nominal[tr]/(prices['PL'][tr]*DD_nominal[(tr,tr)] + (1+tm[tr])*prices['e']*prices['PWM'][tr]*IM_nominal[tr])
    ), tradeables))

## assign parameters
m.tm = Param(m.tradeables, doc='Tax rate on imported commodity tr', initialize=tm, mutable=True)
m.tx = Param(m.tradeables, doc='Tax rate on commodity tr', initialize=tx)

## Calculate derived prices
prices['PD'] = dict(map(lambda tr: (tr, 
    (1+tx[tr])*prices['PL'][tr]
    ), tradeables))
prices['PM'] = dict(map(lambda tr: (tr, 
    prices['e']*prices['PWM'][tr]*(1+tm[tr])*(1+tx[tr])
    ), tradeables))
prices['PC'] = dict(map(lambda tr: (tr, 
    sam(['I',tr,'OTH', 'TOT']) / (sam(['AG', 'WORLD','I', tr]) + sam(['J',tr,'I',tr]))
    ), tradeables))
prices['PWE'] = dict(map(lambda tr: (tr, 
    sam(['WM',tr,'AG', 'WORLD']) / (sam(['J', tr,'WM', tr])/prices['PE'][tr]*prices['e'])
    ), tradeables))
prices['PC'] = dict(map(lambda tr: (tr, 
    sam(['I',tr,'OTH', 'TOT']) / (sam(['AG', 'WORLD','I', tr]) + sam(['J',tr,'I',tr]))
    ), tradeables))
prices['PCI'] = dict(map(lambda j: (j, 
    sum([DI_nominal[(tr, j)]/CI_nominal[j] for tr in tradeables])
    ), sectors))

## Assign price variables
m.e       = Var(doc='Nominal exchange rate (domestic per foreign)', initialize=prices['e'])
m.W       = Var(doc='Wage rate', initialize=prices['W'])
m.PINDEX  = Var(doc='Price index (GDP deflator)', initialize=prices['PINDEX'])
m.R       = Var(m.tradeables, doc='Rental rate of capital in industry tr', initialize=prices['R'])
m.P       = pyo.Var(m.sectors, doc='Price of sector j output (exluding taxes)', initialize=prices['P'])
m.PL      = Var(m.tradeables, doc='Price of commodity tr sold on the local market (excluding taxes)', 
    initialize=prices['PL'])
m.PD      = Var(m.tradeables, doc='Price of commodity tr sold on the local market (including taxes)', 
    initialize=prices['PD'])
m.PE      = Var(m.tradeables, doc='Producer price of exported commodity tr', 
    initialize=prices['PE'])
m.PM      = Var(m.tradeables, doc='Price of imported commodity tr (including duties and taxes)', 
    initialize=prices['PM'])
m.PWE     = Var(m.tradeables, doc='World price of exported product tr (in foreign currency)', 
    initialize=prices['PWE'])
m.PWM     = Var(m.tradeables, doc='World price of imported product tr (in foreign currency)', 
    initialize=prices['PWM'])
m.PC     = Var(m.tradeables, doc='Purchaser price of composite commodity tr (composed of imported and locally produced)', 
    initialize=prices['PC'])
m.PCI    = Var(m.sectors, doc='ntermediate consumption price index of industry j', 
    initialize=prices['PCI'])

# Note: Price of value-added will be added later




### 4.2 Nominal variables

In [ ]:
#note! TO (AG, CAP) FROM (AG, FIRM) 

## Nominal variables
### Taxes/government income
m.DTF           = Var(doc='Receipts from direct taxation on firms\' income', initialize=sam(['AG','GVT','AG','FIRM']))
m.DTH           = Var(m.households, doc='Receipts from direct taxation on household h income',
                    initialize=sam(['AG','GVT','AG',0], over=[households]))
m.TI            = Var(m.tradeables, doc='Receipts from indirect tax on commodity tr',
                    initialize=sam(['AG','GVT','I',0], over=[tradeables]))
m.TIE           = Var(m.tradeables, doc='Receipts from indirect tax on exported commodity tr',
                    initialize=sam(['AG','GVT','WM',0,], over=[tradeables]))
m.TIM           = Var(m.tradeables, doc='Receipts from import duties on commodity tr',
                    initialize=sam(['AG','DUTIES','I',0], over=[tradeables]))

### Incomes
m.TG            = Var(doc='Public transfers to salaried households', initialize=sam(['AG','SAL','AG','GVT']))
m.DIV           = Var(doc='Dividends paid to (capitalist) households', initialize=sam(['AG','CAP','AG','FIRM']))
m.YH            = Var(m.households, doc='Income of type h households',
                    initialize=sam(['OTH','TOT','AG',0], over=[households]))
m.YDH           = Var(m.households, doc='Disposable income of type h households', initialize=dict(map(lambda h: (h, 
                    val(m.YH, h) - val(m.DTH, h)
                    ), households)))
m.YF            = Var(doc='Firms\' income', initialize=sam(['OTH','TOT','AG','FIRM']))
m.YG            = Var(doc='Government income', initialize=sam(['OTH','TOT','AG','GVT']))

### Savings
m.SH            = Var(m.households, doc='Savings of type h households', 
                    initialize=sam(['OTH','ACC','AG',0], over=[households]))
m.SF            = Var(doc='Firms\' savings', initialize=sam(['OTH','ACC','AG','FIRM']))
m.SG            = Var(doc='Government savings', initialize=sam(['OTH','ACC','AG','GVT']))

### Consumption
m.CTH           = Var(m.households, doc='Consumption budget of type h households', initialize=dict(map(lambda h: (h, 
                    val(m.YDH, h) - val(m.SH, h)
                    ), households)))


### Other
m.CAB           = Var(doc='Current account balance', initialize=-1*sam(['OTH', 'ACC','AG','WORLD']))
m.DIV_r         = Var(doc='Dividentd paid to foreigners', initialize=sam(['AG','WORLD','AG','FIRM']))
m.G             = Var(doc='Current public expenditrues', initialize=sam(['I','PUB','AG','GVT']))
m.IT            = Var(doc='Total investment', initialize=sam(['OTH','TOT','OTH','ACC',]))



### 4.3 Volume variables

In [ ]:
## Volume variables
### Trade
m.EX            = Var(m.tradeables, doc='Exports of commodity tr',
                    initialize=sam(['WM',0,'AG','WORLD'], over=[tradeables], prices=prices['PWE']))
m.IM            = Var(m.tradeables, doc='Exports of commodity tr',
                    initialize=sam(['AG','WORLD','I',0], over=[tradeables], prices=prices['PWM']))

### Factors of production
m.LD            = Var(m.sectors, doc='Industry j demand for labour',
                    initialize=sam(['F','LD','J',0], over=[sectors], prices=prices['W']))
m.LS            = Var(doc='Total labor supply', 
                    initialize = sum([val(m.LD, j) for j in sectors]))
m.KD            = Var(m.tradeables, doc='Industry tr demand for capital',
                    initialize=sam(['F','KD','J',0], over=[tradeables], prices=prices['R']))
m.KS            = Var(m.tradeables, doc='Industry tr demand for capital',
                    initialize=sam(['F','KD','J',0], over=[tradeables], prices=prices['R']))


### Supply
m.XS            = Var(m.sectors, doc='Output of industry j',
                    initialize=sam(['OTH','TOT','J',0], over=[sectors], prices=prices['P']))
m.DS            = Var(m.tradeables, doc='Supply of commodity tr on the local market', initialize=dict(map(lambda tr: (tr, 
                    sam(['J',tr,'I',tr], prices=prices['PL'][tr])
                    ), tradeables)))


### Demand/consumption
m.Q             = Var(m.tradeables, doc='Demand for composite commodity tr',
                    initialize=sam(['I',0,'OTH','TOT',], over=[tradeables], prices=prices['PC']))
m.DI            = Var(m.tradeables, m.sectors, doc='Intermediate consumption of commodity tr in industry j',
                    initialize=sam(['I',0,'J',1,], over=[tradeables, sectors], prices=val(m.PC)))
m.CI            = Var(m.sectors, doc='Consumption budget of type h households', initialize=dict(map(lambda j: (j, 
                    sum([val(m.DI, tr, j)*prices['PC'][tr]/prices['PCI'][j] for tr in tradeables])
                    ), sectors)))
m.C             = Var(m.tradeables, m.households, doc='Consumption of commodity i by type h households', 
                    initialize=sam(['I',0,'AG',1], over=[tradeables, households], prices=prices['PC']))
m.DD            = Var(m.tradeables, doc='Demand for domestic commodity tr', initialize=dict(map(lambda tr: (tr, 
                    sam(['J',tr,'I',tr], prices=prices['PL'][tr])
                    ), tradeables)))
m.DIT           = Var(m.tradeables, doc='Total intermediate demand for commodity tr', initialize=dict(map(lambda tr: (tr, 
                    sum([val(m.DI, tr, j) for j in sectors])
                    ), tradeables)))
m.INV           = Var(m.tradeables, doc='Final demand of commodity tr for investment purposes',
                    initialize=sam(['I',0,'OTH','ACC'], over=[tradeables], prices=prices['PC']))


## Value added
value_added     = dict(map(lambda tr: (tr, val(m.LD, tr) + val(m.KD, tr)), tradeables))
value_added['PUB']       = val(m.LD, 'PUB')
m.VA            = Var(m.sectors, doc='Value added in sector j', initialize=value_added)

p_value_added   = dict(map(lambda tr: (tr, 
                    (prices['W']*val(m.LD, tr) + prices['R'][tr]*val(m.KD, tr)) / val(m.VA, tr)
                    ), tradeables))
p_value_added['PUB'] = prices['W']
m.PVA           = Var(m.sectors, doc='Price of industry j value added', initialize=p_value_added)

### Other
m.LEON    = Var(doc='Excess supply on the market for services', initialize=0)

### 4.4 Parameters

In [ ]:
## Production parameters
alpha       = dict(map(lambda tr: (tr, 
                    val(m.W)*val(m.LD, tr) / (val(m.PVA, tr)*val(m.VA, tr))
                    ), tradeables))
m.alpha     = Param(m.tradeables, doc='Elasticity (Cobb-Douglas production function)', 
                    initialize=alpha)
m.A         = Param(m.tradeables, doc='Scale parameter (Cobb-Douglas - Production function)', 
                    initialize=dict(map(lambda tr: (tr, 
                    val(m.VA, tr)/(val(m.LD, tr)**alpha[tr]*val(m.KD, tr)**(1-alpha[tr]))
                    ), tradeables)))
m.aij       = Param(m.tradeables, m.sectors, doc='Coefficient (Leontief - intermediate consumption)', 
                    initialize=dict([((tr, j), val(m.DI, tr, j)/val(m.CI, j)) for tr in tradeables for j in sectors]))
m.v         = Param(m.sectors, doc='Coefficient (Leontief - value added)', 
                    initialize=dict(map(lambda j: (j, 
                    val(m.VA, j)/val(m.XS, j)
                    ), sectors)))
m.io        = Param(m.sectors, doc='Coefficient (Leontief - total intermediate consumption)', 
                    initialize=dict(map(lambda j: (j, 
                    val(m.CI, j)/val(m.XS, j)
                    ), sectors)))

## Distribution parameters
m.gamma     = Param(m.tradeables, m.households, doc='Share of commodity i in type h household consumption budget', 
                    initialize=dict([((tr, h), val(m.PC, tr)*val(m.C, tr, h)/val(m.CTH, h)) for tr in tradeables for h in households]))
lmbda       = sam(['AG','CAP','F','KD']) / sum([sam(['F', 'KD','J', tr]) for tr in tradeables])
m.lmbda     = Param(doc='Share of capital income received by capitalists', initialize=lmbda)
m.lmbda_r   = Param(doc='Share of capital income received by foreigners', 
                    initialize=1 - lmbda - sam(['AG','FIRM','F','KD']) / sum([val(m.R, tr)*val(m.KD, tr) for tr in tradeables]))
m.mu        = pyo.Param(m.sectors, doc='Share of commodity i in total investment expenditures', 
                        initialize=dict(map(lambda tr: (tr, 
                        val(m.PC, tr)*val(m.INV, tr)/val(m.IT)), 
                        tradeables)))
m.psi       = pyo.Param(m.households, doc='Average propensity to save of type h household', 
                        initialize=dict(map(lambda h: (h, 
                        val(m.SH, h)/val(m.YDH, h)), 
                        households)))

## Remaining tax parameters
m.te        = Param(m.tradeables, doc='Tax rate on exported commodity tr', 
                    initialize=dict(map(lambda tr: (tr, 
                    prices['e']*prices['PWE'][tr]/prices['PE'][tr] - 1
                    ), tradeables)))
m.tyh       = pyo.Param(m.households, doc='Direct tax rate on household h income', 
                    initialize=dict(map(lambda h: (h, 
                    val(m.DTH, h)/val(m.YH, h)
                    ), households)))
m.tyf       = pyo.Param(doc='Direct tax on rate of firms\' income', 
                    initialize=val(m.DTF)/val(m.YF))

## Trade parameters
### Elasticities are estimated based on lit review
sigmaE      = { 'AGR': 1.5, 'MAN': 0.5, 'SER': 1.0 }
sigmaM      = { 'AGR': 2.0, 'MAN': 0.6, 'SER': 0.2 }
m.sigmaE    = Param(m.tradeables, doc='Elasticity (CET supply function)', initialize=sigmaE)
m.sigmaM    = Param(m.tradeables, doc='Elasticity (CET demand function)', initialize=sigmaM)

rhoE        = dict(map(lambda tr: (tr, 1/sigmaE[tr] + 1 ), tradeables))
rhoM        = dict(map(lambda tr: (tr, 1/sigmaM[tr] - 1 ), tradeables))
m.rhoE    = Param(m.tradeables, doc='Elasticity parameter (CET supply function)', initialize=rhoE)
m.rhoM    = Param(m.tradeables, doc='Elasticity parameter (CET demand function)', initialize=rhoM)

betaE       = dict(map(lambda tr: (tr, 
                1 / ( (val(m.EX, tr)/val(m.DS, tr))**(1/sigmaE[tr]) / (val(m.PE, tr)/val(m.PL, tr)) + 1 ) 
                ), tradeables))
betaM       = dict(map(lambda tr: (tr, 
                1 / ( (val(m.PD, tr)/val(m.PM, tr)) / (val(m.IM, tr)/val(m.DD, tr))**(1/sigmaM[tr]) + 1) 
                ), tradeables))
m.betaE    = Param(m.tradeables, doc='Distribution parameter (CET supply function)', initialize=betaE)
m.betaM    = Param(m.tradeables, doc='Distribution parameter (CET demand function)', initialize=betaM)

AE          = dict(map(lambda tr: (tr, 
                val(m.XS, tr) / ( betaE[tr]*val(m.EX, tr)**rhoE[tr] + (1-betaE[tr])*val(m.DS, tr)**rhoE[tr])**(1/rhoE[tr])
                ), tradeables))
AM          = dict(map(lambda tr: (tr, 
                val(m.Q, tr) / ( betaM[tr]*val(m.IM, tr)**(-1*rhoM[tr]) + (1-betaM[tr])*val(m.DD, tr)**(-1*rhoM[tr]))**(-1/rhoM[tr])
                ), tradeables))
m.AE    = Param(m.tradeables, doc='Scale parameter (CET - supply function)', initialize=AE)
m.AM    = Param(m.tradeables, doc='Scale parameter (CET - demand function)', initialize=AM)

## 5 Equations

In [ ]:
## Production
def XSEQ(m, j):
    return m.VA[j] == m.v[j]*m.XS[j]
m.XSEQ = pyo.Constraint(m.sectors, rule=XSEQ, doc='Value added demand in industry j (Leontief)')

def CIEQ(m, j):
    return m.CI[j] == m.io[j]*m.XS[j]
m.CIEQ = pyo.Constraint(m.sectors, rule=CIEQ, doc='Total intermediate consumption demand in industry j (Leontief)')

def VAEQ(m, tr):
    return m.VA[tr] == m.A[tr]*(m.LD[tr]**m.alpha[tr])*(m.KD[tr]**(1-m.alpha[tr]))
m.VAEQ = pyo.Constraint(m.tradeables, rule=VAEQ, doc='Cobb-Douglas between labour and capital')

def LDEQ(m, tr):
    return m.W*m.LD[tr] == m.alpha[tr]*m.PVA[tr]*m.VA[tr]
m.LDEQ = pyo.Constraint(m.tradeables, rule=LDEQ, doc='Demand for labour by industry tr')

def LDPEQ(m):
    return m.LD['PUB'] == m.VA['PUB']
m.LDPEQ = pyo.Constraint(rule=LDPEQ, doc='Demand for labour in the public sector')

def KDEQ(m, tr):
    return m.R[tr]*m.KD[tr] == (1-m.alpha[tr])*m.PVA[tr]*m.VA[tr]
m.KDEQ = pyo.Constraint(m.tradeables, rule=KDEQ, doc='Demand for capital by industry j')

def DIEQ(m, tr, j):
    return m.DI[(tr, j)] == m.aij[(tr,j)]*m.CI[j]
m.DIEQ = pyo.Constraint(m.tradeables, m.sectors, rule=DIEQ, doc='Intermediate consumption of commodity i by sector j')


In [ ]:
## Income and savings
def YHSEQ(m):
    return m.YH['SAL'] == m.W*sum(m.LD[j] for j in m.sectors) + m.TG
m.YHSEQ = Constraint(rule=YHSEQ, doc='Household income (workers)')

def YHCEQ(m):
    return m.YH['CAP'] == m.lmbda*sum(m.R[tr]*m.KD[tr] for tr in m.tradeables) + m.DIV
m.YHCEQ = Constraint(rule=YHCEQ, doc='Household income (workers)')

def YDHEQ(m, h):
    return m.YDH[h] == m.YH[h] - m.DTH[h]
m.YDHEQ = Constraint(m.households, rule=YDHEQ, doc='Household disposable income')

def SHEQ(m, h):
    return m.SH[h] == m.psi[h]*m.YDH[h]
m.SHEQ = Constraint(m.households, rule=SHEQ, doc='Household h savings')

def CTHEQ(m, h):
    return m.CTH[h] == m.YDH[h] - m.SH[h]
m.CTHEQ = Constraint(m.households, rule=CTHEQ, doc='Consumption budget for household h')

def YFEQ(m):
    return m.YF == (1-m.lmbda-m.lmbda_r)*sum(m.R[tr]*m.KD[tr] for tr in m.tradeables) 
m.YFEQ = Constraint(rule=YFEQ, doc='Firms income')

def SFEQ(m):
    return m.SF == m.YF - m.DIV - m.DIV_r - m.DTF
m.SFEQ = Constraint(rule=SFEQ, doc='Firms savings')

def YGEQ(m):
    return m.YG == sum(m.TI[tr] + m.TIM[tr] + m.TIE[tr] for tr in m.tradeables) + sum(m.DTH[h] for h in m.households) + m.DTF
m.YGEQ = Constraint(rule=YGEQ, doc='Government income')

def TIEQ(m, tr):
    return m.TI[tr] == m.tx[tr]*(m.PL[tr]*m.DD[tr]+(1+m.tm[tr])*m.e*m.PWM[tr]*m.IM[tr])
m.TIEQ = Constraint(m.tradeables, rule=TIEQ, doc='Receipts from indirect taxation')

def TIMEQ(m, tr):
    return m.TIM[tr] == m.tm[tr]*m.e*m.PWM[tr]*m.IM[tr]
m.TIMEQ = Constraint(m.tradeables, rule=TIMEQ, doc='Receipts from import duties')

def TIEEQ(m, tr):
    return m.TIE[tr] == m.te[tr]*m.PE[tr]*m.EX[tr]
m.TIEEQ = Constraint(m.tradeables, rule=TIEEQ, doc='Receipts from export taxes')

def DTHEQ(m, h):
    return m.DTH[h] == m.tyh[h]*m.YH[h]
m.DTHEQ = Constraint(m.households, rule=DTHEQ, doc='Receipts from income taxes (households)')

def DTFEQ(m):
    return m.DTF == m.tyf*m.YF
m.DTFEQ = Constraint(rule=DTFEQ, doc='Receipts from income taxes (firms)')

def SGEQ(m):
    return m.SG == m.YG - m.G - m.TG
m.SGEQ = Constraint(rule=SGEQ, doc='Government savings')



In [ ]:
## Demand
def CEQ(m, tr, h):
    return m.PC[tr]*m.C[(tr,h)] == m.gamma[(tr,h)]*m.CTH[h]
m.CEQ = Constraint(m.tradeables, m.households, rule=CEQ, doc='Household h consumption of commodity tr')

def INVEQ(m, tr):
    return m.PC[tr]*m.INV[tr] == m.mu[tr]*m.IT
m.INVEQ = Constraint(m.tradeables, rule=INVEQ, doc='Investment in commodity tr')

def DITEQ(m, tr):
    return m.DIT[tr] == sum(m.DI[(tr,j)] for j in m.sectors)
m.DITEQ = Constraint(m.tradeables, rule=DITEQ, doc='Intermediate demand for commodity i')

In [ ]:
## International trade
def XSEEQ(m, tr):
    return m.XS[tr] == m.AE[tr]*(m.betaE[tr]*m.EX[tr]**m.rhoE[tr] + (1-m.betaE[tr])*m.DS[tr]**m.rhoE[tr])**(1/m.rhoE[tr])
m.XSEEQ = Constraint(m.tradeables, rule=XSEEQ, doc='Output from export demand')

def EXEQ(m, tr):
    return m.EX[tr]/m.DS[tr] == ((m.PE[tr]/m.PL[tr])*((1-m.betaE[tr])/m.betaE[tr]))**m.sigmaE[tr]
m.EXEQ = Constraint(m.tradeables, rule=EXEQ, doc='Export demand')

def QEQ(m, tr):
    return m.Q[tr] == m.AM[tr]*(m.betaM[tr]*m.IM[tr]**(-1*m.rhoM[tr]) + (1-m.betaM[tr])*m.DD[tr]**(-1*m.rhoM[tr]))**(-1/m.rhoM[tr])
m.QEQ = Constraint(m.tradeables, rule=QEQ, doc='Domestic consumption of composite good')

def IMEQ(m, tr):
    return m.IM[tr]/m.DD[tr] == ((m.PD[tr]/m.PM[tr])*(m.betaM[tr]/(1-m.betaM[tr])))**sigmaM[tr]
m.IMEQ = Constraint(m.tradeables, rule=IMEQ, doc='Import demand')

def CABEQ(m):
    return m.CAB == sum([m.e* (m.PWE[tr]*m.EX[tr] - m.PWM[tr]*m.IM[tr]) - m.lmbda_r*m.R[tr]*m.KD[tr] for tr in tradeables]) - m.DIV_r
m.CABEQ = Constraint(rule=CABEQ, doc='Current account balance')

In [ ]:
print('Q',  val(m.Q))
print('AM', AM)
print('betaM', betaM)
print('IM', val(m.IM))
print('rhoM', rhoM)
print('sigmaM', sigmaM)
print('DD', val(m.DD))

In [ ]:
## Prices
def PVAPEQ(m):
    return m.PVA['PUB'] == m.W
m.PVAPEQ = Constraint(rule=PVAPEQ, doc='Equivalence between PVA and W for public sector')

def PCIEQ(m, j):
    return m.PCI[j]*m.CI[j] == sum(m.PC[tr]*m.DI[(tr,j)] for tr in tradeables)
m.PCIEQ = Constraint(m.sectors, rule=PCIEQ, doc='Intermediate consumption price index')

def PCEQ(m, j):
    return m.P[j]*m.XS[j] == m.PVA[j]*m.VA[j] + m.PCI[j]*m.CI[j]
m.PCEQ = Constraint(m.sectors, rule=PCEQ, doc='Production costs for sector j')

def PDEQ(m, tr):
    return m.PD[tr] == m.PL[tr]*(1+m.tx[tr])
m.PDEQ = Constraint(m.tradeables, rule=PDEQ, doc='Price of commodity tr including taxes')

def PMEQ(m, tr):
     return m.PM[tr] == m.e*m.PWM[tr]*(1+m.tm[tr])*(1+m.tx[tr])
m.PMEQ = Constraint(m.tradeables, rule=PMEQ, doc='Price of imports')

def PEEQ(m, tr):
     return m.PE[tr]*(1+m.te[tr]) == m.e*m.PWE[tr]
m.PEEQ = Constraint(m.tradeables, rule=PEEQ, doc='Price of exports')

def PQEQ2(m, tr):
    return m.PC[tr]*m.Q[tr] == m.PD[tr]*m.DD[tr] + m.PM[tr]*m.IM[tr]
m.PQEQ2 = Constraint(m.tradeables, rule=PQEQ2, doc='Price of composite good on local market')

def PEQ(m, tr):
    return m.P[tr]*m.XS[tr] == m.PL[tr]*m.DS[tr] + m.PE[tr]*m.EX[tr]
m.PEQ = Constraint(m.tradeables, rule=PEQ, doc='Price of output')

#note use of baseline value_added and p_value_added here
def PINDEXEQ(m):
    return m.PINDEX == sum([m.PVA[j]*value_added[j] for j in sectors]) / sum([p_value_added[j]*value_added[j] for j in sectors])
m.PINDEXEQ = Constraint(rule=PINDEXEQ, doc='Price index')




In [ ]:
## Equilibrium
def QEQ2(m, bns):
    return m.Q[bns] == sum(m.C[(bns, h)] for h in m.households) + m.DIT[bns] + m.INV[bns]
m.QEQ2 = Constraint(m.goods, rule=QEQ2, doc='Domestic absorption (over goods)')

def PPUBEQ(m):
    return m.G == m.P['PUB']*m.XS['PUB']
m.PPUBEQ = Constraint(rule=PPUBEQ, doc='Equilibrium on the market for public services')

def DSEQ(m, tr):
    return m.DS[tr] == m.DD[tr]
m.DSEQ = Constraint(m.tradeables, rule=DSEQ, doc='Local commodity market equilibrium')

def WEQ(m):
    return m.LS == sum(m.LD[j] for j in m.sectors)
m.WEQ = Constraint(rule=WEQ, doc='Labour market equilibrium')

def REQ(m, tr):
    return m.KS[tr] == m.KD[tr]
m.REQ = Constraint(m.tradeables, rule=REQ, doc='Capital market equilibrium')

def ITEQ(m):
    return m.IT == sum(m.SH[h] for h in m.households) + m.SF + m.SG - m.CAB
m.ITEQ = Constraint(rule=ITEQ, doc='Investment-savings equilibrium')

## Other
def WALRAS(m):
    return m.LEON == m.Q['SER'] - sum(m.C[('SER', h)] for h in m.households) - m.DIT['SER'] - m.INV['SER']
m.WALRAS =  Constraint(rule=WALRAS, doc="Verification of Walras' law")


## 6 Create instances

### 6.1 Helper functions

In [ ]:
def count_nonfixed(i, active=True):
    
    num_non_fixed = 0
    num_fixed = 0
    num_vars = 0
    for block in i.block_data_objects(active=active):
        var_set = ComponentSet()
        for c in block.component_data_objects(
                ctype=Constraint, active=True, descend_into=True):
            for v in identify_variables(c.body):
                var_set.add(v)

        for v in var_set:
            if v.is_fixed():
                num_fixed += 1
            else: 
                num_non_fixed += 1
        num_vars = len(var_set)
 
    return num_non_fixed, num_fixed, num_vars

def count_constraints(i, active=True):
    num_constraints = 0
    for block in i.block_data_objects(active=active):
        for data in block.component_map(pyo.Constraint, active=active).values():
            for key in data._data.keys():
                num_constraints += 1
    return num_constraints

def checkSquareness(inst):
    num_non_fixed, num_fixed, num_vars = count_nonfixed(inst)
    if (num_non_fixed == inst.nconstraints()):
        print('Instance IS SQUARE.')
    else:
        print('Instance is NOT SQUARE!')

def checkFeasibility(inst, limit=1e-9):
    feasible = True
    num_checked = 0
    for block in inst.block_data_objects(active=True):
        for constraint in block.component_map(pyo.Constraint, active=True).values():
            # print(constraint, len(list(constraint._data.keys())), constraint.doc)
            for index in constraint.index_set():
                if index in list(constraint._data.keys()):
                    num_checked = num_checked + 1
                    if abs(constraintValue(constraint[index])) > limit:
                        feasible = False
                        print('Infeasibility:', constraint, index, constraintValue(constraint[index]))
    print(f'Checked {num_checked} constraints.')
    
    if feasible:
        print('All equations balanced. Instance is feasible')

def has_value(obj, key):
    if obj[key].value != 0:
         if obj[key].value != 0.0:
             if obj[key].value is not None:
                 return True
    return False

In [ ]:
def applyFixes(inst, m):
    inst.e.fix() # numeraire
    inst.DIV.fix()
    inst.DIV_r.fix()
    inst.G.fix()
    inst.KS.fix()
    inst.LS.fix()
    inst.TG.fix()
    inst.CAB.fix()
    inst.PWE.fix()
    inst.PWM.fix()

### 6.2 Define optimization objective

In [ ]:
def obj_expression(m):
    return sum(m.C[(tr, h)] for tr,h in m.tradeables*m.households)

m.OBJ = pyo.Objective(rule=obj_expression, sense=pyo.maximize)

### 6.3 Create base instance

In [ ]:
inst_base = m.create_instance()
applyFixes(inst_base, m)


In [ ]:
for block in inst_base.block_data_objects(active=True):
    print('variables:          ', idaes_utils.model_statistics.number_variables_in_activated_equalities(block))
    print('  fixed:            ', idaes_utils.model_statistics.number_fixed_variables_in_activated_equalities(block))
    print('  unfixed:          ', idaes_utils.model_statistics.number_unfixed_variables_in_activated_equalities(block))
    print('constraints:        ', idaes_utils.model_statistics.number_activated_equalities(block))
    print('degrees of freedom: ', idaes_utils.model_statistics.degrees_of_freedom(block))
    checkSquareness(block)
    checkFeasibility(block)
    idaes_utils.model_statistics.report_statistics(block)


In [ ]:
# Display some of the parameters
inst_base.A.display()
inst_base.alpha.display()
inst_base.io.display()
inst_base.v.display()
inst_base.aij.display()
inst_base.gamma.display()
inst_base.psi.display()
inst_base.mu.display()
inst_base.lmbda.display()

### 6.4 Create scenarios

In [ ]:
## simulations
inst_scenario_1 = m.create_instance()
applyFixes(inst_scenario_1, m)

## note: adjusted parameters must have  mutable=True
### 50% reduction in import tarrifs on manufactured goods:
inst_scenario_1.tm['MAN'] = 0.5 * inst.tm['MAN']

## 7 Solve instances

In [ ]:
opt = pyo.SolverFactory('ipopt')
opt.options['max_cpu_time'] = 120 #seconds
opt.options['warm_start_init_point'] = 'yes'
opt.options['halt_on_ampl_error'] = 'yes'

In [ ]:
res_base = opt.solve(inst_base, tee=True)
if (res_base.Solver.Status != 'ok'):
    print('Status NOT ok!')
if (res_base.Solver[0]['Termination condition'] != 'optimal'):
    print('Optimal solution NOT found!')
else:
    print('Optimal solution found!')
res_base

In [ ]:
res_scenario_1 = opt.solve(inst_scenario_1, tee=False)
if (res_scenario_1.Solver.Status != 'ok'):
    print('Status NOT ok!')
if (res_scenario_1.Solver[0]['Termination condition'] != 'optimal'):
    print('Optimal solution NOT found!')
else:
    print('Optimal solution found!')
res_scenario_1

## 8 Compare solutions

### 8.1 Helper functions

In [ ]:
def compare_scenarios(instances_dict):
    results_obj = {}    
    for instance_key in instances_dict.keys():
        for block in instances_dict[instance_key].block_data_objects(active=True):
            var_set = ComponentSet()
            for v in block.component_map(pyo.Var, active=True).values():
                if not v._name in results_obj.keys():
                    results_obj[v._name] = {}
                for key in v.keys():
                    if not key in results_obj[v._name].keys():
                        results_obj[v._name][key] = {}
                    results_obj[v._name][key][instance_key] = v[key].value

    return results_obj

def results_dict_to_pandas_dict(results_obj):
    pandas_dict = {}
    for key in results_obj.keys():
        df = pd.DataFrame.from_dict(results_obj[key], orient='index')
        columns = list(df.columns)
        for column in columns:
            if column == columns[0]:
                continue
            df[f'{column} %diff'] = df[column]/df[columns[0]] - 1
            df[f'{column} %diff'] = np.where((df[column] == 0) & (df[columns[0]] == 0), 
                                             0.0, df[f'{column} %diff'])
        pandas_dict[key] = df
    return pandas_dict

def write_pandas_dict_to_file(pandas_dict, wb_path):
    # book = load_workbook(wb_path)
    writer = pd.ExcelWriter(wb_path)
    workbook = writer.book
    percent_fmt = workbook.add_format({'num_format': '0.00%'})
    wrapped_text = workbook.add_format({
        'text_wrap': True
    })
    
    # writer.book = book
    for key in pandas_dict.keys():
        df = pandas_dict[key]
        first_row = 2
        last_row = 1 + len(df)
        num_index_cols = 1
        if type(df.index) == pd.core.indexes.multi.MultiIndex:
            num_index_cols = len(df.index[0])
        base_col = num_index_cols + 1
        num_scenarios = int((len(list(df.columns)) - 1) / 2)
        
        df.to_excel(writer, sheet_name=key, index=True)
        
        workbook = writer.book
        worksheet = writer.sheets[key]
        
        # Apply a conditional format to the cell range.
        worksheet.conditional_format(
            f'{chr(64 + base_col + num_scenarios + 1)}{first_row}:{chr(64 + base_col + num_scenarios*2)}{last_row}', 
            {'type': '3_color_scale'})
        worksheet.set_column(f'{chr(64 + base_col + num_scenarios + 1)}:{chr(64 + base_col + num_scenarios*2)}', None, percent_fmt)
        worksheet.set_row(0, 30, wrapped_text)
        
    writer.save()
    print('done.')

### 8.2 Store comparisons

In [ ]:
results_obj = compare_scenarios({
    'base': inst_base, 
    'scenario 1': inst_scenario_1
    })
    

In [ ]:
pandas_dict = results_dict_to_pandas_dict(results_obj)

In [ ]:
write_pandas_dict_to_file(pandas_dict, 'output/results_pyomo.xlsx')